### fetch intraday currency options data

In [1]:
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import json 

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta, time
import time as Time


while datetime.now().time() < time(17, 5, 30):
    Time.sleep(1)

samco=StocknoteAPIPythonBridge()
login=json.loads(samco.login(body={"userId":'********','password':'********@9998','yob':'1998'}))
samco.set_session_token(sessionToken=login['sessionToken'])


currency_list = ['EURINR','GBPINR','JPYINR','USDINR']


for currency in currency_list:

    option_chain = pd.DataFrame(json.loads(samco.get_option_chain(search_symbol_name=currency,exchange=samco.EXCHANGE_CDS))['optionChainDetails'])

    symbols = option_chain[option_chain['lastTradedPrice']!='0.0000']['tradingSymbol'].unique()

    main_data = pd.DataFrame()

    start_date = str(date.today())+' 09:00:00'
    end_date = str(date.today())+' 17:00:00'

    for i in symbols:
        print(i)

        try:
            data = pd.DataFrame(json.loads(samco.get_intraday_candle_data(symbol_name=i,exchange=samco.EXCHANGE_CDS, from_date=start_date, to_date=end_date))['intradayCandleData'])
            data['symbol']=i   #'NIFTY05JAN2318300CE'
            main_data = pd.concat([main_data,data],axis=0)
            
            main_data['dateTime'] = pd.to_datetime(main_data['dateTime'])
            main_data.sort_values(by='dateTime',inplace=True)

        except:
            continue


    path = fr'E:/option_data/CURRENCY/{currency}/2024/APR/data_'
    dates = path+str(date.today())+'.csv'

    main_data.to_csv(dates,index=False)
    
samco.logout()

KeyError: 'optionChainDetails'

In [ ]:
import os
os.system("shutdown /s /t 1")

#### Move file one folder to another 

In [2]:
import shutil

currency_list = ['EURINR','GBPINR','JPYINR','USDINR']
dates = ['2024-02-05']

for currency in currency_list:
    path = fr'E:\option_data\CURRENCY\{currency}\2024\JAN\data_'
    
    for D in dates:
        source_path = path+str(D)+'.csv'

        destination_path = fr'E:/option_data/CURRENCY/{currency}/2024/FEB/'

        # Use shutil.move to move the file
        shutil.move(source_path, destination_path)

Error: Destination path 'E:/option_data/CURRENCY/EURINR/2024/FEB/data_2024-02-05.csv' already exists

### fetch intraday MCX options data

In [2]:
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import json 

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta, time
import time as Time
import pytz

samco=StocknoteAPIPythonBridge()
login=json.loads(samco.login(body={"userId":'********','password':'********@11','yob':'1984'}))
samco.set_session_token(sessionToken=login['sessionToken'])


MCX_list = ['GOLD','SILVER','CRUDEOIL','NICKEL','COPPER','NATURALGAS','ZINC']


for metal in MCX_list:

    option_chain = pd.DataFrame(json.loads(samco.get_option_chain(search_symbol_name=metal,exchange=samco.EXCHANGE_MCX))['optionChainDetails'])

    symbols = option_chain[option_chain['lastTradedPrice']!='0.00']['tradingSymbol'].unique()
    
    if len(symbols)==0:
        continue
    
    main_data = pd.DataFrame()
    
    day = date.today()
    start_date = str(day)+' 09:00:00'
    end_date = str(day)+' 23:30:00'

    for i in symbols:
        print(i)

        try:
            data = pd.DataFrame(json.loads(samco.get_intraday_candle_data(symbol_name=i,exchange=samco.EXCHANGE_MCX, from_date=start_date, to_date=end_date))['intradayCandleData'])
            data['symbol']=i   #'NIFTY05JAN2318300CE'
            main_data = pd.concat([main_data,data],axis=0)
        except:
            continue


    main_data['dateTime'] = pd.to_datetime(main_data['dateTime'])
    main_data.sort_values(by=['dateTime'],inplace=True)

    path = f'E:\option_data\MCX\{metal}\data_'
    dates = path+str(date.today())+'.csv'

#     main_data.to_csv(dates,index=False)

KeyError: 'sessionToken'

In [11]:
import pandas as pd
import os

path = 'D:\speculatio\EMA Samco_Tradesheet'
broker_account_path = r'D:\speculatio\EMA Samco_Tradesheet\account_details\Account_Details.csv'
accountdetails = pd.read_csv(broker_account_path)

files =  [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
final_data = pd.DataFrame()

for F in files:
    
    user_name = F.split('_trade')[0]
    trade_df = pd.read_csv(f"{path}\{F}",header=None)
    if trade_df.iloc[-1,2]=='BUY':
        trade_df = trade_df[:-1]

    user_account_detail = accountdetails[accountdetails['user_name']==user_name]
    initial_capital = user_account_detail['initial_capital'].iloc[0]
    # start_date = user_account_detail['start_date'].iloc[0]

    df = pd.DataFrame(columns=range(12))

    j=0

    for i in range(0,len(trade_df),2):

        df.at[j] = trade_df.iloc[i]

        try:
            if (trade_df.iloc[i,[4,5]] == trade_df.iloc[(i+1),[4,5]]).all() and (trade_df.iloc[i,2]=='BUY') and (trade_df.iloc[(i+1),2]=='SELL'):

                df.at[j,6] = trade_df.iloc[(i+1),3]
                df.at[j,7] = (df.iloc[j,6] - df.iloc[j,3])
                df.at[j,8] =  (df.at[j,7])* int(df.iloc[j,5])
                df.at[j,9] = round(df.iloc[j,7]/df.iloc[j,3]*100,2)
                df.at[j,10] = (3.4*df.iloc[j,5])+40
                df.at[j,11] = round((df.iloc[j,7]-(df.at[j,10]/int(df.iloc[j,5])))/df.iloc[j,3]*100,2)
                df.at[j,12] = initial_capital + sum(df[8]) - sum(df[10])

                j+=1

        except IndexError:
            pass


    df.rename(columns={0:'Date Time',1:'Symbol',2:'Trade Type',3:'Buy Price',4:'Status',5:'Qty',6:'Sell Price',7:'PnL',8:'Total PnL',9:'Gross PnL %',10:'Expenses',11:'Net PnL %',12:'Final capital'},inplace=True)
    df.fillna(0,inplace=True)
    
    df['user'] = user_name
    df['result'] = df['Net PnL %'].apply(lambda x: 'win' if x>0 else 'loss')

    for i in range(len(df)):
        df.at[i, 'DD'] = round((df.loc[i, 'Final capital'] / pd.concat([df.loc[:i, 'Final capital'], pd.Series(initial_capital)]).max() - 1) * 100,2)

    df.dropna(inplace=True)
    df['Trade_no'] = df.index
    
    final_data = pd.concat([final_data,df])

In [12]:
final_data

,Date Time,Symbol,Trade Type,Buy Price,Status,Qty,Sell Price,PnL,Total PnL,Gross PnL %,Expenses,Net PnL %,Final capital,user,result,DD,Trade_no
0,01-Jun-2023 10:30:00,NIFTY23JUNFUT,BUY,18634.40,Complete,50.0,18602.30,-32.10,-1605.0,-0.17,210.0,-0.19,998185.0,Akshat_Mamta,loss,-0.18,0
1,02-Jun-2023 09:29:59,NIFTY23JUNFUT,BUY,18640.00,Complete,50.0,18571.10,-68.90,-3445.0,-0.37,210.0,-0.39,994530.0,Akshat_Mamta,loss,-0.55,1
2,02-Jun-2023 13:30:00,NIFTY23JUNFUT,BUY,18662.90,Complete,50.0,18610.00,-52.90,-2645.0,-0.28,210.0,-0.31,991675.0,Akshat_Mamta,loss,-0.83,2
3,05-Jun-2023 09:30:05,NIFTY23JUNFUT,BUY,18704.90,Complete,50.0,18641.60,-63.30,-3165.0,-0.34,210.0,-0.36,988300.0,Akshat_Mamta,loss,-1.17,3
4,07-Jun-2023 09:15:04,NIFTY23JUNFUT,BUY,18712.10,Complete,50.0,18724.05,11.95,597.5,0.06,210.0,0.04,988687.5,Akshat_Mamta,win,-1.13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,21-Nov-2023 09:30:01,NIFTY23NOVFUT,BUY,19833.00,Complete,50.0,19765.85,-67.15,-3357.5,-0.34,210.0,-0.36,518320.0,Rajesh,loss,-1.70,49
50,22-Nov-2023 14:30:00,NIFTY23NOVFUT,BUY,19848.00,Complete,50.0,19821.90,-26.10,-1305.0,-0.13,210.0,-0.15,516805.0,Rajesh,loss,-1.99,50
51,24-Nov-2023 11:30:01,NIFTY23NOVFUT,BUY,19856.40,Complete,50.0,19827.40,-29.00,-1450.0,-0.15,210.0,-0.17,515145.0,Rajesh,loss,-2.31,51
52,24-Nov-2023 14:30:00,NIFTY23NOVFUT,BUY,19836.50,Complete,50.0,19852.80,16.30,815.0,0.08,210.0,0.06,515750.0,Rajesh,win,-2.19,52
